# Import Library

In [1]:
import re
import json
import urllib.request
import datetime as dt
import pandas   as pd
import numpy    as np
pd.set_option('display.max_colwidth', -1)
import requests

from bs4            import BeautifulSoup
from urllib.request import urlopen
from inscriptis     import get_text
from lxml           import html

# Extract Category  & Sub Category
![X](image/Category_SubCategory.png)

In [2]:
page = urlopen('https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/layout_v2/stocks/stocklisting.js')
source = page.read()
soup = BeautifulSoup(source,'html.parser') # Parse the HTML as a string

mk_row=[]
mk_row.clear()

no_of_row = len(soup.find_all('strong'))

for row in range(no_of_row):
    
    #print(row)
    
    # Get row wise XML
    v_row = soup.find_all('tr')[row]
    # Convert into BeautifulSoup
    get_row_wise_xml = BeautifulSoup(str(v_row),'lxml')
    
    v_main_category = get_row_wise_xml.find_all('strong')
    v_sub_category = get_row_wise_xml.find_all('p')
   
    for sub_cat_row in range(len(v_sub_category)):        
       
        v_sub_cat_url  = v_sub_category[sub_cat_row].find_all('a')
    
        mk_row.append([ str(v_main_category[0].text).replace(':','') ,
                        v_sub_category[sub_cat_row].text,
                        str(v_sub_cat_url[0]['id']).replace('\\"','')])


In [20]:
mk_row

[['Main Market', 'Health Care', 'main_healthcare'],
 ['Main Market', 'Energy', 'main_energy'],
 ['Main Market', 'Technology', 'main_technology'],
 ['Main Market', 'Property', 'main_property'],
 ['Main Market', 'Utilities', 'main_utilities'],
 ['Main Market', 'Finance Services', 'main_finance'],
 ['Main Market', 'Telecommunications & Media', 'main_telcomedia'],
 ['Main Market', 'Consumer Products & Services', 'main_consumer'],
 ['Main Market', 'Construction', 'main_construction'],
 ['Main Market', 'Real Estate Investment Trusts', 'main_reits'],
 ['Main Market', 'Industrial Products & Services', 'main_indprod'],
 ['Main Market', 'Plantation', 'main_plantation'],
 ['Main Market', 'Transportation & Logistics', 'main_transport'],
 ['Main Market',
  'Special Purpose Acquisition Company',
  'main_specialpurposeact'],
 ['Main Market', 'Closed End Fund', 'main_closedfund'],
 ['Ace Market', 'Health Care', 'ace_healthcare'],
 ['Ace Market', 'Technology', 'ace_technology'],
 ['Ace Market', 'Transp

In [3]:
df_main = pd.DataFrame(mk_row,columns=['Main_Category','Sub_Category','URL_Extension'])
df_main.head()

,Main_Category,Sub_Category,URL_Extension
0,Main Market,Health Care,main_healthcare
1,Main Market,Energy,main_energy
2,Main Market,Technology,main_technology
3,Main Market,Property,main_property
4,Main Market,Utilities,main_utilities


# Reading Child table URL by Subcategory
![X](image/URL.png)

In [22]:
#https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/layout/stocks/data.js

# df_main['URL_for_Invidual']=df_main.apply(lambda X: str('https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/json/sectors/')
#                                           +str(X.Main_Category).lower().replace(' ','_')
#                                           +'/'
#                                           +str(X.Sub_Category).lower().replace(' ','')
#                                           +str('/stocks.js'), axis =1)

################### Reading the Child table to extract the company name ###############
page = urlopen('https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/layout/stocks/data.js')
source = page.read()
soup = BeautifulSoup(source,'html.parser')
v_js_string = str(soup)

full_url=[]
full_url.clear()

# Reading dataFrameto get the full URL per URL_Extension
for index, row in df_main.iterrows():
    
    # Split line based on URL_Extension and get the URL
    for item in v_js_string.split("break"):
        # Checking the line containing the URL extension
        if row["URL_Extension"] in item:
            full_url.append(re.findall('https.*js',item.strip())[0])
            
# Adding the full URL as column in DataFrame
df_main['URL_for_Invidual']=full_url            

In [5]:
df_main.head()

df_main.to_csv('Main_cat.csv')

## Extract all the company name under each subcategory
![X](image/individual_summary1.png)

In [23]:
mk_lst=[]
mk_lst.clear()
logdate = dt.datetime.now()


for index, row in df_main.iterrows():
    url = row['URL_for_Invidual']
    html = urllib.request.urlopen(url).read().decode('utf-8') 
    
    v_text = get_text(html)    
    v_text = v_text.replace('var stockData = ','')
    v_text = v_text.replace(';','')
    js = json.loads(v_text)
    
    for i in range(sum(map(len, js.values()))):
        
        mk_lst.append ([index,
                       js['Stocks'][i]['counter'],
                       js['Stocks'][i]['open'],
                       js['Stocks'][i]['high'],
                       js['Stocks'][i]['low'],
                       js['Stocks'][i]['lastdone'],
                       js['Stocks'][i]['change'],
                       js['Stocks'][i]['volume'],
                       js['Stocks'][i]['percentchange'],
                       js['Stocks'][i]['stockcode'],
                       logdate
                      ])

df_daily_summary_per_comp_log = pd.DataFrame(mk_lst,columns=['fkey','symbol','open','high','low','lastdone','change','volume','percentchange','stockcode','logdate'])


In [8]:
df_daily_summary_per_comp_log.head()

,fkey,symbol,open,high,low,lastdone,change,volume,percentchange,stockcode,logdate
0,0,ADVENTA,0.360,0.360,0.350,0.355,-0.015,300,-4.05,7191,2019-02-27 22:04:36.703325
1,0,AHEALTH,8.670,8.740,8.650,8.700,0.150,82,1.75,7090,2019-02-27 22:04:36.703325
2,0,CCMDBIO,1.140,1.160,1.140,1.140,0.000,"5,677",0.00,7148,2019-02-27 22:04:36.703325
3,0,HARTA,5.160,5.170,5.050,5.120,-0.040,"44,313",-0.78,5168,2019-02-27 22:04:36.703325
4,0,IHH,5.500,5.680,5.500,5.660,-0.040,"37,233",-0.70,5225,2019-02-27 22:04:36.703325


In [9]:
df_main.shape

(45, 4)

# Extract individual details
![X](image/individual_details.png)

In [24]:
# Web Crawler class

class AppCrawler:
    
    v_name         = ''
    v_board        = ''
    v_stock_code   = ''
    v_52_week_high = ''
    v_52_week_low  = ''
    v_open         = ''
    v_high         = ''
    v_low          = ''
    v_last         = ''
    v_chg          = ''
    v_chg_pct      = ''
    v_Vol_00       = ''
    v_buy_Vol_00   = ''
    v_sell_Vol_00  = ''
    v_update       = ''
        
    def __init__(self, starting_url, depth):
        self.starting_url = starting_url
        self.depth = depth
        self.apps = []        

    def crawl(self):
        self.get_app_from_link(self.starting_url)
        return

    def get_app_from_link(self, link):
        page = urlopen(link)
        source = page.read()
        soup = BeautifulSoup(source,'html.parser') 
        
        ################################# Decoding all the TAG ##########################
        
        AppCrawler.v_name         = soup.findAll(id = 'slcontent_0_ileft_0_compnametxt')[0].text

        ##  Sample TAG for board, Stock Code, 52 week High, Low ##
        #
        # [<li class="f14"><span>Board</span> : Main</li>,
        #  <li class="f14"><span>Stock Code</span> : 5218</li>,
        #  <li class="f14"><span>52 Week High</span> : 0.910</li>,
        #  <li class="f14"><span>52 Week Low</span> : 0.255</li>]

        AppCrawler.v_board        = str(soup.findAll('li',{'class':"f14"})[0].text).rsplit(':')[1].strip()
        AppCrawler.v_stock_code   = str(soup.findAll('li',{'class':"f14"})[1].text).rsplit(':')[1].strip()
        AppCrawler.v_52_week_high = str(soup.findAll('li',{'class':"f14"})[2].text).rsplit(':')[1].strip()
        AppCrawler.v_52_week_low  = str(soup.findAll('li',{'class':"f14"})[3].text).rsplit(':')[1].strip()

        AppCrawler.v_open         = soup.findAll(id = 'slcontent_0_ileft_0_opentext')[0].text
        AppCrawler.v_high         = soup.findAll(id = 'slcontent_0_ileft_0_hightext')[0].text
        AppCrawler.v_low          = soup.findAll(id = 'slcontent_0_ileft_0_lowtext')[0].text
        AppCrawler.v_last         = soup.findAll(id = 'slcontent_0_ileft_0_lastdonetext')[0].text
        AppCrawler.v_chg          = soup.findAll(id = 'slcontent_0_ileft_0_chgtext')[0].text
        AppCrawler.v_chg_pct      = soup.findAll(id = 'slcontent_0_ileft_0_chgpercenttrext')[0].text
        AppCrawler.v_Vol_00       = soup.findAll(id = 'slcontent_0_ileft_0_voltext')[0].text
        AppCrawler.v_buy_Vol_00   = soup.findAll(id = 'slcontent_0_ileft_0_buyvol')[0].text
        AppCrawler.v_sell_Vol_00  = soup.findAll(id = 'slcontent_0_ileft_0_sellvol')[0].text
        AppCrawler.v_update       = str(soup.findAll(id = 'slcontent_0_ileft_0_datetxt')[0].text).split(':')[1].strip().replace(' |','') \
                                    + ' '\
                                    + str(soup.findAll(id = 'slcontent_0_ileft_0_timetxt')[0].text)

        return

In [25]:
mk_list=[]
mk_list.clear()
logdate = dt.datetime.now()

print('Running....')

# Get number of rows in the source data frame
no_of_record = df_daily_summary_per_comp_log.shape[0]

for index, row in df_daily_summary_per_comp_log.iterrows():
    
    v_url = 'https://www.thestar.com.my/business/marketwatch/stocks/?qcounter='+row['symbol'] # symbol is the column name from dataframe
    crawler = AppCrawler(v_url, 0)
    crawler.crawl()
    
    mk_list.append([row['fkey'],
                    row['symbol'],
                    crawler.v_name,
                    crawler.v_board,
                    crawler.v_stock_code,
                    crawler.v_52_week_high,
                    crawler.v_52_week_low,
                    crawler.v_open,
                    crawler.v_high,
                    crawler.v_low,
                    crawler.v_last,
                    crawler.v_chg,
                    crawler.v_chg_pct,
                    crawler.v_Vol_00,
                    crawler.v_buy_Vol_00,
                    crawler.v_sell_Vol_00,
                    crawler.v_update,
                    logdate
                   ]                    
                  )
    
    # Monitor the progress of the extraction
    if np.mod(index,100)==0:
        print(str(dt.datetime.now())+' : '+str(np.round(((index+1)/no_of_record)*100,2)) +'% Completed' )

df_daily_extraction = pd.DataFrame(mk_list, columns=['fkey','symbol','name','board','stock_code','52_week_high','52_week_low','open','high','low','last','chg','chg_pct','Vol_00','buy_Vol_00','sell_Vol_00','update','logdate'])
df_daily_extraction.to_csv('Extraction/Star_Online_Stock_'+'{0:%Y%m%d}'.format(dt.datetime.now())+'.csv',index=False,date_format='%Y-%m-%d %H:%M:%S')

print('Done....')


Running....
2019-03-01 23:42:14.630603 : 0.06% Completed
2019-03-01 23:44:31.327234 : 5.73% Completed
2019-03-01 23:46:14.352753 : 11.41% Completed
2019-03-01 23:48:08.637035 : 17.08% Completed
2019-03-01 23:49:59.935370 : 22.76% Completed
2019-03-01 23:51:44.006293 : 28.43% Completed
2019-03-01 23:53:22.975057 : 34.11% Completed
2019-03-01 23:54:59.247392 : 39.78% Completed
2019-03-01 23:56:28.451195 : 45.46% Completed
2019-03-01 23:58:07.520586 : 51.14% Completed
2019-03-02 00:00:04.516145 : 56.81% Completed
2019-03-02 00:01:43.796369 : 62.49% Completed
2019-03-02 00:03:07.719676 : 68.16% Completed
2019-03-02 00:05:05.114704 : 73.84% Completed
2019-03-02 00:06:48.533478 : 79.51% Completed
2019-03-02 00:09:10.527036 : 85.19% Completed
2019-03-02 00:11:37.437974 : 90.86% Completed
2019-03-02 00:14:01.372455 : 96.54% Completed
Done....


In [261]:
df_daily_extraction.head()

,fkey,symbol,name,board,stock_code,52_week_high,52_week_low,open,high,low,last,chg,chg_pct,Vol_00,buy_Vol_00,sell_Vol_00,update,logdate
0,0,ADVENTA,ADVENTA BHD,Main,7191,0.625,0.350,0.365,0.370,0.365,0.370,0.020,5.71,288,0.365 / 40,0.370 / 102,25 Feb 2019 7:11 PM,2019-02-25 21:48:09.930637
1,0,AHEALTH,APEX HEALTHCARE BHD,Main,7090,8.870,4.930,8.800,8.820,8.660,8.800,0.000,0.00,130,8.720 / 5,8.880 / 9,25 Feb 2019 7:11 PM,2019-02-25 21:48:09.930637
2,0,CCMDBIO,CCM DUOPHARMA BIOTECH BHD,Main,7148,3.430,0.925,1.140,1.160,1.130,1.160,0.030,2.65,"10,924",1.150 / 505,1.160 / 155,25 Feb 2019 7:11 PM,2019-02-25 21:48:09.930637
3,0,HARTA,HARTALEGA HOLDINGS BHD,Main,5168,11.880,4.920,5.250,5.370,5.230,5.280,0.030,0.57,"31,697","5.280 / 3,317",5.310 / 230,25 Feb 2019 7:11 PM,2019-02-25 21:48:09.930637
4,0,IHH,IHH HEALTHCARE BERHAD,Main,5225,6.420,4.540,5.790,5.790,5.710,5.750,-0.050,-0.86,"30,388","5.750 / 1,232",5.760 / 290,25 Feb 2019 7:11 PM,2019-02-25 21:48:09.930637
